<a href="https://colab.research.google.com/github/AlexeyEvzrezov/rfm_abc_xyz_analysis/blob/main/rfm_abc_xyz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from pycbrf.toolbox import ExchangeRates
import datetime as dt

np.set_printoptions(precision=2)

# Functions

## Preprocessing

In [ ]:
def preprocess(df):
    
    # invoice no, invoice daste, customer, customer, payer, amount, cuurency
    df.columns = ['no', 'date', 'cust', 'contr', 'amount', 'cur'] 

    df.date = pd.to_datetime(df.date)
    df.amount = df.amount.str.replace(',', '.')
    df.amount = df.amount.str.replace('\xa0', '') 
    df.amount = df.amount.astype('float')  

    df['year'] = df.date.dt.year
    df['year_month'] = df['date'].dt.strftime('%Y-%m')
    
    for curr in ['EUR', 'USD']:
        curr_sales = df[df.cur==curr]
        for row in curr_sales.index:
            rate = ExchangeRates(df.loc[row].date)[curr][-1]
            rate = float(rate)
            df.at[row, 'amount'] = df.loc[row].amount * rate

    df = df.dropna()
    
    return df

## RFM

In [ ]:
def rfm_score(row, col, q):
    
    if row <= q.loc[.25, col]:
        return 1 if col == 'recency' else 4
    elif row <= q.loc[.5, col]:
        return 2 if col == 'recency' else 3
    elif row <= q.loc[.75, col]:
        return 3 if col == 'recency' else 2
    else:
        return 4 if col == 'recency' else 1


def create_rfm(df):
    now = df.date.max()
    aggs = {'date': lambda x: (now - x.max()).days, 
            'no': lambda x: len(x), 
            'amount': lambda x: x.sum()}

    rfm = df.groupby('cust').agg(aggs).reset_index()
    rfm['date'] = rfm['date'].astype(int)
    rfm.columns = ['cust', 'recency', 'frequency', 'monetary']

    quantiles = rfm.quantile([0.25, 0.5, 0.75])

    for col in rfm.columns[-3:]:
        rfm[col + '_score'] = rfm[col].apply(rfm_score, col=col, q=quantiles)

    rfm['rfm_score'] = rfm.iloc[:, -3:].astype('str').sum(axis=1).astype('uint') 
    rfm = rfm.sort_values(by=['rfm_score'])
    return rfm   

## ABC

In [ ]:
def create_abc(df, split=[0.8, 0.15, 0.05]):
    
    out = df.groupby('cust').amount.sum().reset_index()
    out = out.sort_values(by=['amount'], ascending=False, ignore_index=True)
    total_sales = out.amount.sum()
    out['share'] = out.amount / total_sales
    out['cum_share'] = out.share[0]
    for i in range(1, out.shape[0]):
        out.cum_share[i] = out.cum_share[i - 1] + out.share[i] 

    out['abc'] = 'B'
    out.loc[out.cum_share <= split[0], 'abc'] = 'A'
    out.loc[out.cum_share > 1 - split[2], 'abc'] = 'C'
    
    return out

## XYZ

In [ ]:
def create_xyz(df, split=[0.1, 0.25]): 
    
    out = df.groupby('cust').amount.sum().reset_index()

    monthly_sales = pd.pivot_table(df, 'amount', 'year_month', 'cust', 'sum')
    monthly_sales = monthly_sales.fillna(0)

    for customer in monthly_sales.columns:
        monthly_cust = monthly_sales[customer].values
        sigma = monthly_cust.std()
        mu = monthly_cust.mean()
        out.loc[out.cust == customer, 'cv'] = sigma / mu

    out = out.sort_values(by=['cv'], ignore_index=True)
   
    out['xyz'] = 'Y'
    out.loc[out.cv <= split[0], 'xyz'] = 'X'
    out.loc[out.cv > split[1], 'xyz'] = 'C'
    
    return out